In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

from utils import load

In [30]:
import os

In [32]:
solution_path = '../solution/baseline/predictions.csv'
prediction = pd.read_csv(solution_path, index_col=0)
_, target = load(1, 'test-target')
np.sqrt(mean_squared_error(prediction['prediction'], target['target']))

11.449004887101944

### Base Line
- 1: 11.449004887101944
- 

### Trees
- 1:


In [41]:
solution_path = '../solution/trees/regression/1.csv'
prediction = pd.read_csv(solution_path, index_col=0)
_, target = load(1, 'test-target')
np.sqrt(mean_squared_error(prediction['prediction'], target['target']))

9.7385090284560576

- 4

In [57]:
from sklearn.metrics import roc_auc_score

solution_path = '../solution/trees/classification/4.csv'
prediction = pd.read_csv(solution_path, index_col=0)
_, target = load(4, 'test-target')
roc_auc = roc_auc_score(target['target'], prediction['prediction'])
roc_auc

0.76980390068060445